- Average Annual Household Transportation Costs: Out-of-pocket expenses including tolls, parking, and transit fares

    - These costs do not include other vehicle operating and purchase costs.

    - Costs are annualized using factor of 320


In [1]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '${:0,.0f}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [3]:
# person data
person = pd.read_csv(output_path / 'agg/dash/person_cost.csv')
# household data
hh = pd.read_csv(output_path / 'agg/dash/hh_geog.csv')

annual_factor = summary_config['weekday_to_annual']
# list of equity geographies
equity_geogs = summary_config['equity_geogs']
not_equity_geogs = ["NOT in " + item for item in equity_geogs]

In [4]:
df_person = person.copy()
df_person['is_rgc'] = "In RGC"
df_person.loc[df_person['hh_rgc'] == 'Not in RGC', 'is_rgc'] = "Not in RGC"
# included modes
# TODO: ask what this means
df_person = df_person[(df_person['mode'].isin(['SOV','HOV2','HOV3+']) & (df_person['dorp'] == 1)) 
                      | (~df_person['mode'].isin(['SOV','HOV2','HOV3+']))]

df_hh = hh.copy()
df_hh['is_rgc'] = "In RGC"
df_hh.loc[df_hh['hh_rgc'] == 'Not in RGC', 'is_rgc'] = "Not in RGC"

In [5]:
def annual_costs(geog):
    df = df_person.groupby(geog, as_index=False)['travcost_wt'].sum().set_index(geog)
    df['hhexpfac'] = df_hh.groupby(geog)['hhexpfac'].sum()

    df.loc['Region'] = [df_person['travcost_wt'].sum(), df_hh['hhexpfac'].sum()]
    df['Average Annual Costs'] = (df['travcost_wt'] / df['hhexpfac']) * annual_factor

    return df[['Average Annual Costs']]

In [6]:
annual_costs('hh_county')

,Average Annual Costs
hh_county,
King,"$2,421"
Kitsap,"$2,881"
Outside Region,"$4,953"
Pierce,"$2,826"
Snohomish,"$2,999"
Region,"$2,638"


In [7]:
annual_costs('is_rgc')

,Average Annual Costs
is_rgc,
In RGC,$969
Not in RGC,"$2,815"
Region,"$2,638"


In [8]:
annual_costs('hh_rgc')

,Average Annual Costs
hh_rgc,
Auburn,"$1,897"
Bellevue,"$1,000"
Bothell Canyon Park,"$2,429"
Bremerton,"$1,182"
Burien,"$1,843"
Everett,"$1,113"
Federal Way,"$1,978"
Greater Downtown Kirkland,"$1,853"
Kent,"$1,708"


In [9]:
annual_costs('hh_rg_proposed')

,Average Annual Costs
hh_rg_proposed,
CitiesTowns,"$3,452"
Core,"$2,568"
HCT,"$2,794"
Metro,"$1,715"
UU,"$3,236"
Region,"$2,638"


In [10]:
# get total intersections by equity geography
df1 = df_person[equity_geogs].apply(lambda x: x * df_person['travcost_wt']).sum().reset_index()
df1.columns = ['Equity Group', 'travcost_wt']
df1.set_index('Equity Group', inplace=True)
df1['hhexpfac'] = df_hh[equity_geogs].apply(lambda x: x * df_hh['hhexpfac']).sum()

df1['Inside Equity Geography'] = (df1['travcost_wt'] / df1['hhexpfac']) * annual_factor

# outside equity geographies
df2 = df_person[equity_geogs].apply(lambda x: (1-x) * df_person['travcost_wt']).sum().reset_index()
df2.columns = ['Equity Group', 'travcost_wt']
df2.set_index('Equity Group', inplace=True)
df2['hhexpfac'] = df_hh[equity_geogs].apply(lambda x: (1-x) * df_hh['hhexpfac']).sum()

# add outside equity geographies costs
df1['Outside Equity Geography'] = (df2['travcost_wt'] / df2['hhexpfac']) * annual_factor

df1[['Inside Equity Geography','Outside Equity Geography']]

,Inside Equity Geography,Outside Equity Geography
Equity Group,,
hh_disability_reg,"$2,522","$2,735"
hh_elderly_reg,"$2,789","$2,501"
hh_english_reg,"$2,542","$2,694"
hh_poverty_reg,"$2,321","$2,849"
hh_racial_reg,"$2,380","$2,855"
hh_youth_reg,"$3,115","$2,247"
hh_poverty_50,"$1,720","$2,658"
hh_racial_50,"$2,365","$2,761"
